# Minimizing TPD with pygmo

In [1]:
import pygmo as pg
from gibbs.models.ceos import PengRobinson78
import numpy as np
import attr

## Case with TPD

In [2]:
@attr.s(auto_attribs=True)
class WhitsonModel:
    z: np.ndarray
    P: float
    T: float
    Tc: np.ndarray
    Pc: np.ndarray
    acentric_factor: np.ndarray
    bip: np.ndarray

    @property
    def model(self):
        return PengRobinson78(
            z=self.z, 
            Tc=self.Tc, 
            Pc=self.Pc, 
            acentric_factor=self.acentric_factor, 
            bip=self.bip
        )

    @property
    def number_of_components(self):
        return len(self.z)

    def fugacity(self, P, T, z):
        Z_factor = self.calculate_Z(P, T, z)
        return self.model.calculate_fugacity(P, T, z, Z_factor)

    def calculate_Z(self, P, T, z):
        Z_factor = self.model.calculate_Z_minimal_energy(P, T, z)
        return Z_factor

class tpd_problem:
    
    def __init__(self):
        self.z = np.array([0.5, 0.42, 0.08])
        self.omegas = np.array([0.0115, 0.1928, 0.4902])
        self.Tcs = np.array([190.556, 425.16667, 617.666667])
        self.Pcs = np.array([4604318.9, 3796942.8, 2.096e6])
        self.kijs = np.zeros((3, 3))
        self.P = 3.447e6
        self.T = 410.928
        self.model = WhitsonModel(
            z=self.z, 
            P=self.P, 
            T=self.T,
            Tc=self.Tcs,
            Pc=self.Pcs,
            acentric_factor=self.omegas, 
            bip=self.kijs
        )
    
    def fitness(self, x):
        f_z = self.model.fugacity(self.P, self.T, self.z)
        f_x = self.model.fugacity(self.P, self.T, x)
        tpd = np.sum(x * (np.log(f_x / f_z)))
        
        tol = 1e-3
        if not 1 - tol <= np.sum(x) <= 1 + tol:
            # Penalization if candidate is not in feasible space
            penalty_parameter = 1e3
            tpd += penalty_parameter * self._penalty_function(x, 1)
            
        return [tpd]
    
    def get_bounds(self):
        return ([0] * len(self.z), [1] * len(self.z))
    
    def _penalty_function(self, x, feasible_limit):
        feasible_region_marker = np.abs(x.sum() - feasible_limit)
        return feasible_region_marker * feasible_region_marker

In [3]:
prob = pg.problem(tpd_problem())
print(prob)

Problem name: <class '__main__.tpd_problem'>
	Global dimension:			3
	Integer dimension:			0
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [0, 0, 0]
	Upper bounds: [1, 1, 1]

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 0

	Thread safety: none



In [4]:
algo = pg.algorithm(pg.sade(variant=6))  # best/1/bin
pop = pg.population(prob = tpd_problem(), size=500)
pop = algo.evolve(pop)

print(pop)

Problem name: <class '__main__.tpd_problem'>
	Global dimension:			3
	Integer dimension:			0
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [0, 0, 0]
	Upper bounds: [1, 1, 1]

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 1000

	Thread safety: none

Population size: 500

List of individuals: 
#0:
	ID:			16244895529788346808
	Decision vector:	[0.480056, 0.0449946, 0.181527]
	Fitness vector:		[86.1506]
#1:
	ID:			7771597265982225595
	Decision vector:	[0.0605341, 0.366756, 0.294772]
	Fitness vector:		[76.7695]
#2:
	ID:			4809392223095965319
	Decision vector:	[0.219481, 0.969532, 0.836214]
	Fitness vector:		[1051.67]
#3:
	ID:			16333267765516729594
	Decision vector:	[0.579322, 0.925833, 0.445997]
	Fitness vector:		[906.115]
#4:
	ID:			12782731649702115059
	Decision vector:	[0.616578, 0.233824, 0.6

In [5]:
print(f"tpd = {pop.champion_f[0]}")
print(f"x = {pop.champion_x}")
print(f"sum(x) = {pop.champion_x.sum()}")

tpd = -0.7676663918774027
x = [0.0471922  0.18953837 0.76882139]
sum(x) = 1.0055519581851504
